# <center> Практическая работа №8.
### <center> Проверка гипотез о законе распределения, проблема нормализации выборки Сравнение характеристик распределения двух генеральных совокупностей параметрическими методами 

In [1]:
#Загрузка основных модулей:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn
import plotly.graph_objects as go
%matplotlib inline
!pip install odfpy

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

**Задание 1.** В файле «Данные 8_1» приведены данные успеваемости
студентов 1-го курса по четырем математическим дисциплинам по 100
балльной шкале (нормированные к 1). Иван и Петр, входящие в число
этих студентов, поспорили насчет того, по каким дисциплинам у их однокурсников хороших оценок больше, а по каким меньше. Иван думает,
что в этом плане между дисциплинами особой разницы нет, а Петр
настаивает, что есть. Подумав, они решили, что для объективного разрешения спора им надо, опираясь на методы математической статистики, сопоставить вероятности получения не менее n баллов в разных парах дисциплин (а именно, проверить гипотезы о равенстве вероятностей
получения не менее n баллов по этим дисциплинам (см. п. 5 подраздела
8.2)). Ваша задача - помочь Ивану и Петру, взяв на себя анализ пары дисциплин (в соответствии с вашим вариантом). Нужно найти диапазоны значений n, при которых на уровне значимости а, данные подтверждают мнение Ивана и диапазоны значений n, при которых на уровне
значимости а, данные подтверждают мнение Петра.
Замечание. Данные реальные и требуют предварительной обработки


*Вариант 3:* ОМА и ДУ


In [2]:
# Загружаем данные из файла Excel
data = pd.read_excel("Data_8_1.xlsx", names=['DU_itog', 'MA', 'OMA', 'AandG'])
data = data.drop(index=0)  # Удаляем первую строку, если она не содержит данных

# Извлекаем данные по дисциплинам ОМА и ДУ и приводим их к типу float
scores_OMA = list(data['OMA'].dropna().astype(float))
scores_DU = list(data['DU_itog'].dropna().astype(float))

Будем следовать алгоритму: Пусть некоторое событие $А$ в серии из $n_1$ испытаний произошло $k_1$ раз, а в другой серии из $n_2$ испытаний произошло $k_2$ раза.  Найдем эти величины из данной нам выборки

In [3]:
# Определяем количество наблюдений в каждой дисциплине
num_OMA = np.size(scores_OMA) #n_1
num_DU = np.size(scores_DU) #n_2
total_count = num_OMA + num_DU #n

# Инициализируем списки для хранения результатов
success_counts_OMA = []
success_counts_DU = []
min_h_statistics = []

Произведу проверку условий: 

1) Пусть соблюдены условия: сумма $(n_1 +n2)$ - «большая» и наименьшая из величин $\frac {(k_1+k_2)n_1}{n1+n2}$, $\frac {(k_1+k_2)n_2}{n1+n2}$, $\frac {(n - k_1-k_2)n_1}{n1+n2}$, $\frac {(n - k_1-k_2)n_2}{n1+n2}$ больше 5;

2) Пусть соблюдены условия: $n_1 +n_2 >20$ и наименьшая из величин $\frac {(k_1+k_2)n_1}{n1+n2}$, $\frac {(k_1+k_2)n_2}{n1+n2}$, $\frac {(n - k_1-k_2)n_1}{n1+n2}$, $\frac {(n - k_1-k_2)n_2}{n1+n2}$ больше 3.

От выполнения этих условий будет зависеть выбор статистики:

1) $Z = \frac{\frac{k_1}{n_1}- \frac{k_2}{n_2}}{\sqrt{h(1-h)(\frac{1}{n_1}+\frac{1}{n_2})}}, h = \frac{k_1+k_2}{n_1+n_2}$

2) $Z = \frac{n_1+n_2}{n_1 \cdot n_2 \cdot k_1 \cdot k_2}$

In [4]:
# Перебираем значения от 1 до 0 с шагом 0.01
for threshold in np.arange(1, 0, -0.01):
    # Считаем количество студентов, получивших не менее 'threshold' баллов
    count_OMA = sum(1 for score in scores_OMA if score >= threshold)
    count_DU = sum(1 for score in scores_DU if score >= threshold)
    
    success_counts_OMA.append(count_OMA)
    success_counts_DU.append(count_DU)
    
    # Рассчитываем минимальные статистики h
    min_h_statistic = np.min([(count_OMA + count_DU) * num_OMA / (num_OMA + num_DU),
                               (count_OMA + count_DU) * num_DU / (num_OMA + num_DU),
                               (total_count - count_OMA - count_DU) * num_OMA / (num_OMA + num_DU),
                               (total_count - count_OMA - count_DU) * num_DU / (num_OMA + num_DU)])
    
    min_h_statistics.append(min_h_statistic)
# Массив для хранения индексов выбранных статистик
selected_statistics = []

# Заполняем массив по условиям выбора статистик
for h in min_h_statistics:
    if h < 3:
        selected_statistics.append(0)  # Статистика незначимого различия
    elif h < 5:
        selected_statistics.append(2)  # Статистика сравнительного анализа
    else:
        selected_statistics.append(1)  # Статистика значимого различия

# Массив для выборочных значений статистик
z_scores = []

# Вычисляем статистики z
for i in range(len(min_h_statistics)):
    if selected_statistics[i] == 1:
        h_i = (success_counts_OMA[i] + success_counts_DU[i]) / total_count
        z_i = (success_counts_OMA[i] / num_OMA - success_counts_DU[i] / num_DU) / \
               np.sqrt(h_i * (1 - h_i) * (1 / num_OMA + 1 / num_DU))
        z_scores.append(z_i)
    elif selected_statistics[i] == 2:
        z_i = (num_OMA + num_DU) / (num_OMA * num_DU * success_counts_OMA[i] * success_counts_DU[i])
        z_scores.append(z_i)
    else:
        z_scores.append(np.NaN)

Далее найдем при каких значениях подтверждается чья гипотеза

In [5]:
# Массив p-значений
p_values = []
for i in range(len(selected_statistics)):
    if selected_statistics[i] == 1:
        p_value_i = 2 * np.min([sts.norm(0, 1).cdf(z_scores[i]), 1 - sts.norm(0, 1).cdf(z_scores[i])])
        p_values.append(p_value_i)
    elif selected_statistics[i] == 2:
        p_value_i = 2 * np.min([sts.chi2(1).cdf(z_scores[i]), 1 - sts.chi2(1).cdf(z_scores[i])])
        p_values.append(p_value_i)
    else:
        p_values.append(np.NaN)

# Массив значений порога n (в данном случае это значения от 1 до 0 с шагом 0.01)
thresholds = np.arange(1, 0, -0.01)

# Массивы значений порога, при которых гипотеза Ивана и Петра принимаются
accepted_n_Ivan = []  # Приемлемые n для гипотезы Ивана
accepted_n_Petr = []  # Приемлемые n для гипотезы Петра

# Заполняем массивы соответствующими значениями
for i in range(len(selected_statistics)):
    if p_values[i] > 0.1:  # Уровень значимости для гипотезы Ивана
        accepted_n_Ivan.append(round(thresholds[i] * 100))
    if p_values[i] <= 0.1:  # Уровень значимости для гипотезы Петра
        accepted_n_Petr.append(round(thresholds[i] * 100))

# Выводим результаты анализа
print(f'Гипотеза Ивана подтверждается при n, равных: {accepted_n_Ivan}')
print(f'Гипотеза Петра подтверждается при n, равных: {accepted_n_Petr}')

Гипотеза Ивана подтверждается при n, равных: [50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 19]
Гипотеза Петра подтверждается при n, равных: [95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3]


**Задание 2.** Сгенерируйте две выборки $(А и В)$ различных объемов из нормально распределенных генеральных совокупностей с параметрами $m_A, \sigma_A$ и $m_B, \sigma_B$ (возьмите значения $m_A$ и $m_B$ близкими, но не
равными, а $\sigma_A$ и $\sigma_B$ выберите такими, чтобы близки (но не равны)
были их квадраты). Проверьте гипотезу о равенстве дисперсий по критерию Фишера (см. п. 2 подраздела 8.2) на уровне значимости 0,05.

Сформулируйте выводы.
Если гипотеза подтвердится, то сгенерируйте две другие выборки
(С и D), для которых гипотеза отклонится (при необходимости поварьируйте параметры, поэкспериментируйте, чтобы добиться заданного
результата).
Если гипотеза о равенстве дисперсий выборок А и В будет отклонена, то сгенерируйте две другие выборки (С' и D'), для которых
гипотеза подтвердится (при необходимости поварьируйте параметры,
поэкспериментируйте, чтобы добиться заданного результата).

In [6]:
def lab8_2(ma, sa, mb, sb):
    # Уровень значимости
    alpha = 0.05
    # Размеры выборок
    na = 2500  # Размер первой выборки
    nb = 5000  # Размер второй выборки

    # Генерация выборок из нормального распределения
    A = np.random.normal(ma, sa, na)  # Выборка A с параметрами (среднее, стандартное отклонение, размер)
    B = np.random.normal(mb, sb, nb)  # Выборка B

    # Вычисление выборочной дисперсии для каждой выборки (используем ddof=1 для несмещенной оценки)
    Sa = np.var(A, ddof=1)  # Дисперсия выборки A
    Sb = np.var(B, ddof=1)  # Дисперсия выборки B

    # Вывод выборочной дисперсии
    print('Sa = ', Sa)
    print('Sb =', Sb)

    # Вычисление статистики F
    F = Sa / Sb
    # Находим критические значения для F-распределения
    D1 = sts.f(na - 1, nb - 1).ppf(alpha / 2)  # Нижняя граница критической области
    D2 = sts.f(na - 1, nb - 1).ppf(1 - alpha / 2)  # Верхняя граница критической области

    # Вывод результатов
    print('Выборочное значение дисперсии:', F)
    print('G:(-inf,', D1,')∪(',D2,',inf)')  # Вывод критической области

    # Проверка гипотезы об равенстве дисперсий
    if D1 < F < D2:
        print('Гипотеза принимается')  # Если значение F попадает в интервал, гипотеза принимается
    else:
        print('Гипотеза отвергается')  # Если значение F вне интервала, гипотеза отвергается

In [7]:
lab8_2(1, 3.3, 1.1, 3.1)

Sa =  11.155218416937998
Sb = 9.509294131394507
Выборочное значение дисперсии: 1.173085852935135
G:(-inf, 0.9339720224562511 )∪( 1.069861540181964 ,inf)
Гипотеза отвергается


In [8]:
lab8_2(1, 3.05, 1.05, 3.005)

Sa =  9.440990021246483
Sb = 9.084769604537055
Выборочное значение дисперсии: 1.0392107265473773
G:(-inf, 0.9339720224562511 )∪( 1.069861540181964 ,inf)
Гипотеза принимается


**Задание 3.** Для выполнения задания используйте пару выборок,
смоделированных при выполнении задания 2, для которой гипотеза о
равенстве дисперсий подтвердилась. Проверьте гипотезу о равенстве
математических ожиданий. Укажите диапазон значений $\alpha$, при которых гипотеза принимается.

In [9]:
na = 2500
nb = 5000
ma = 1
sa = 3.05
mb = 1.05
sb = 3.005
A = np.random.normal(ma, sa, na)
B = np.random.normal (mb, sb, nb)

st = sts.ttest_ind(A, B, equal_var=True, axis=0) [0]
p = sts. ttest_ind(A, B, equal_var=True, axis=0) [1]
print ( 'Выборочное значение статистики =', st)
print ('p =', p)
print ('При alpha <= ', p,'гипотеза принимается')

Выборочное значение статистики = -1.0159773951070297
p = 0.3096729327716854
При alpha <=  0.3096729327716854 гипотеза принимается


**Задание 4.** Для выполнения этого задания используйте пару выборок, смоделированных при выполнении задания 2, для которой гипотеза
о равенстве дисперсий была отклонена. Проверьте гипотезу о равенстве
математических ожиданий. Укажите диапазон значений а, при которых гипотеза принимается.

In [10]:
na = 2500
nb = 5000
ma = 1
sa = 3.3
mb = 1.1
sb = 3.1
A = np.random.normal(ma, sa, na)
B = np.random.normal(mb, sb, nb)

st = sts.ttest_ind(A, B, equal_var=True, axis=0) [0]
p = sts. ttest_ind(A, B, equal_var=True, axis=0) [1]
print ('Выборочное значение статистики =', st)
print ('p =', p)
print ('При alpha <= ', p, ' гипотеза принимается')

Выборочное значение статистики = -1.3511639562642164
p = 0.17668368507802693
При alpha <=  0.17668368507802693  гипотеза принимается
